In [41]:
#Import Data
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os
import glob
import datetime
import prophet #will attempt time series analysis


In [47]:
#List all csv files only
csv_files = glob.glob('data/*.{}'.format('csv'))

In [35]:
df = pd.concat([pd.read_csv(f, on_bad_lines='skip', encoding = 'unicode_escape') for f in csv_files ], ignore_index=True)  #data quality is not very good...
df

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,16/08/03,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,16/08/03,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,16/08/03,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,16/08/03,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,16/08/03,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7126,E0,22/05/2022,Crystal Palace,Man United,1.0,0.0,H,1.0,0.0,H,...,2.04,0.25,1.68,2.15,1.74,2.23,1.88,2.25,1.74,2.16
7127,E0,22/05/2022,Leicester,Southampton,4.0,1.0,H,0.0,0.0,D,...,2.63,-0.75,1.83,2.07,1.88,2.03,1.94,2.26,1.87,2.01
7128,E0,22/05/2022,Liverpool,Wolves,3.0,1.0,H,1.0,1.0,D,...,3.28,-2.50,2.02,1.77,2.06,1.83,2.19,1.99,2.07,1.80
7129,E0,22/05/2022,Man City,Aston Villa,3.0,2.0,H,0.0,1.0,A,...,3.36,-2.25,2.06,1.84,2.05,1.86,2.09,2.03,2.01,1.87


In [37]:
#way too many columns
df.shape

(7131, 151)

In [39]:
df['Date'].value_counts() #Also, need to Clean and Reformat Date

22/05/2022    10
26/12/05      10
26/12/08      10
11/05/14      10
12/05/2019    10
              ..
28/02/05       1
14/09/2015     1
27/09/2015     1
28/09/2015     1
17/08/2015     1
Name: Date, Length: 1964, dtype: int64

In [43]:
#First focus on the ones that don't have the full year
df['Date'].str.len().value_counts()

8.0     4850
10.0    2280
Name: Date, dtype: int64

In [46]:
df.loc[(df['Date'].str.len() == 8)] #More than half the DF

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,AvgC<2.5,AHCh,B365CAHH,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA
0,E0,16/08/03,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,16/08/03,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,16/08/03,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,16/08/03,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,16/08/03,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5226,E0,21/05/17,Liverpool,Middlesbrough,3.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5227,E0,21/05/17,Man United,Crystal Palace,2.0,0.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5228,E0,21/05/17,Southampton,Stoke,0.0,1.0,A,0.0,0.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5229,E0,21/05/17,Swansea,West Brom,2.0,1.0,H,0.0,1.0,A,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### Possible Solution: Split out the Date Column First

In [65]:
df[['date1', 'date2', 'date3']] = df['Date'].str.split('/', expand=True)

In [66]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,date1,date2,date3
0,E0,16/08/03,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,03
1,E0,16/08/03,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,03
2,E0,16/08/03,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,03
3,E0,16/08/03,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,03
4,E0,16/08/03,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,03


In [69]:
#For the dates missing the full year, add the '20' in front of it
df.loc[df.date3.str.len()==2, 'date3'] = '20' + df['date3']
#This works because we're only looking at dates from >= 2003

In [71]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,date1,date2,date3
0,E0,16/08/03,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
1,E0,16/08/03,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
2,E0,16/08/03,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
3,E0,16/08/03,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
4,E0,16/08/03,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003


#### Also need to change dd/mm/yyyy (which most of Europe uses) to mm/dd/yyyy

In [74]:
#df['new_col'] = 
df['Date'] = df[['date2','date1','date3']].apply(lambda row: '/'.join(row.values.astype(str)), axis=1)

In [75]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,date1,date2,date3
0,E0,08/16/2003,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
1,E0,08/16/2003,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
2,E0,08/16/2003,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
3,E0,08/16/2003,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
4,E0,08/16/2003,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003


In [82]:
df['date2'].isna().sum() #missing 

1

In [83]:
df[df['date2'].isna()] #can safely ignore this row

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,date1,date2,date3
4470,NaN,nan/nan/nan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [84]:
df = df[df['date2'].notna()]


In [85]:
#Still need to check if there is a month value greater than 12 that is in the date2 column, which should be month
df[df['date2'].astype(int)>12] #none, we are good to go with dates

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,date1,date2,date3


In [86]:
df.head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,B365CAHA,PCAHH,PCAHA,MaxCAHH,MaxCAHA,AvgCAHH,AvgCAHA,date1,date2,date3
0,E0,08/16/2003,Arsenal,Everton,2.0,1.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
1,E0,08/16/2003,Birmingham,Tottenham,1.0,0.0,H,1.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
2,E0,08/16/2003,Blackburn,Wolves,5.0,1.0,H,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
3,E0,08/16/2003,Fulham,Middlesbrough,3.0,2.0,H,1.0,1.0,D,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
4,E0,08/16/2003,Leicester,Southampton,2.0,2.0,D,2.0,0.0,H,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16,08,2003
